In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train = pd.read_csv(r"C:\Users\USER\Desktop\csv files\image_bins_all_train.csv")
test = pd.read_csv(r"C:\Users\USER\Desktop\csv files\image_bins_all_test.csv")

In [4]:
print("***** Train_Set *****")
print(train.head())
print("\n")
print("***** Test_Set *****")
print(test.head())

***** Train_Set *****
   Bins0  Bins1  Bins2  Bins3  Bins4  Bins5  Bins6  Bins7  Class
0   5345     12      0      0      0     58      0  15601      1
1   6582      2      0      0      0    130      0  13024      1
2   7612      0      0      0     50    127      0  16419      1
3   5145     12      0      0      0    259      0  13044      1
4   4065      0      0      0      4    850      0   9815      1


***** Test_Set *****
   Bins0  Bins1  Bins2  Bins3  Bins4  Bins5  Bins6  Bins7  Class
0   4065      0      0      0      4    850      0   9815      1
1   4029      0      0      0     43   1101      0  10167      1
2   3829      0      0      0     14    451      0  13794      1
3   3211      0      0      0     53    282      0  11794      1
4   4238      0      0      0      8    116      0  11368      1


In [5]:
#You can get some initial statistics of both the train and test DataFrames using pandas' describe() method.
print("***** Train_Set *****")
print(train.describe())
print("\n")
print("***** Test_Set *****")
print(test.describe())

***** Train_Set *****
              Bins0         Bins1         Bins2         Bins3         Bins4  \
count  13998.000000  13998.000000  13998.000000  13998.000000  13998.000000   
mean    5182.125589      4.435848      0.043578      0.084298      7.257394   
std     2267.763800     18.973895      0.772714      2.841387     27.864997   
min      756.000000      0.000000      0.000000      0.000000      0.000000   
25%     3816.000000      0.000000      0.000000      0.000000      0.000000   
50%     4697.000000      0.000000      0.000000      0.000000      0.000000   
75%     5886.000000      1.000000      0.000000      0.000000      1.000000   
max    57620.000000    678.000000     61.000000    192.000000   1745.000000   

              Bins5         Bins6         Bins7         Class  
count  13998.000000  13998.000000  13998.000000  13998.000000  
mean     102.010787      0.487927  12520.972710      0.500000  
std      199.491070      4.957744   2824.913221      0.500018  
min       

In [6]:
print(train.columns.values)

['Bins0' 'Bins1' 'Bins2' 'Bins3' 'Bins4' 'Bins5' 'Bins6' 'Bins7' 'Class']


In [7]:
#It is very important to note that not all machine learning algorithms support missing values in the data that you are feeding to them. K-Means being one of them. So we need to handle the missing values present in the data. Let's first see where are the values missing:
# For the train set
train.isna().head()

,Bins0,Bins1,Bins2,Bins3,Bins4,Bins5,Bins6,Bins7,Class
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False


In [8]:
# For the test set
test.isna().head()

,Bins0,Bins1,Bins2,Bins3,Bins4,Bins5,Bins6,Bins7,Class
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False


In [9]:
#Let's get the total number of missing values in both datasets.
print("*****In the train set*****")
print(train.isna().sum())
print("\n")
print("*****In the test set*****")
print(test.isna().sum())
#There are no missing values in this dataset

*****In the train set*****
Bins0    0
Bins1    0
Bins2    0
Bins3    0
Bins4    0
Bins5    0
Bins6    0
Bins7    0
Class    0
dtype: int64


*****In the test set*****
Bins0    0
Bins1    0
Bins2    0
Bins3    0
Bins4    0
Bins5    0
Bins6    0
Bins7    0
Class    0
dtype: int64


In [10]:
train[['Bins0', 'Bins1']].groupby(['Bins0'], as_index=False).mean().sort_values(by='Bins1', ascending=False)

,Bins0,Bins1
3506,7381,603.000000
2951,6271,498.000000
2868,6145,463.000000
3563,7526,293.000000
1725,4583,246.666667
...,...,...
2726,5903,0.000000
2736,5916,0.000000
2738,5922,0.000000
2740,5925,0.000000


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13998 entries, 0 to 13997
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Bins0   13998 non-null  int64
 1   Bins1   13998 non-null  int64
 2   Bins2   13998 non-null  int64
 3   Bins3   13998 non-null  int64
 4   Bins4   13998 non-null  int64
 5   Bins5   13998 non-null  int64
 6   Bins6   13998 non-null  int64
 7   Bins7   13998 non-null  int64
 8   Class   13998 non-null  int64
dtypes: int64(9)
memory usage: 984.4 KB


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Bins0   10002 non-null  int64
 1   Bins1   10002 non-null  int64
 2   Bins2   10002 non-null  int64
 3   Bins3   10002 non-null  int64
 4   Bins4   10002 non-null  int64
 5   Bins5   10002 non-null  int64
 6   Bins6   10002 non-null  int64
 7   Bins7   10002 non-null  int64
 8   Class   10002 non-null  int64
dtypes: int64(9)
memory usage: 703.4 KB


In [21]:
feature_df = train[['Bins0', 'Bins1', 'Bins2','Bins3', 'Bins4', 'Bins5', 'Bins6', 'Bins7']]
X = np.asarray(feature_df)

In [22]:
y = np.array(train['Class'])

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13998 entries, 0 to 13997
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Bins0   13998 non-null  int64
 1   Bins1   13998 non-null  int64
 2   Bins2   13998 non-null  int64
 3   Bins3   13998 non-null  int64
 4   Bins4   13998 non-null  int64
 5   Bins5   13998 non-null  int64
 6   Bins6   13998 non-null  int64
 7   Bins7   13998 non-null  int64
 8   Class   13998 non-null  int64
dtypes: int64(9)
memory usage: 984.4 KB


In [24]:
kmeans = KMeans(n_clusters=2) # You want cluster the passenger records into 2: Survived or Not survived
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [25]:
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print(correct/len(X))

0.5109301328761252


In [26]:
kmeans = kmeans = KMeans(n_clusters=2, max_iter=600, algorithm = 'auto')
kmeans.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=600,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [27]:
KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=600,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=600,
       n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [28]:
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print(correct/len(X))

0.5109301328761252


In [29]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [30]:
kmeans.fit(X_scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=600,
       n_clusters=2, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [31]:
correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print(correct/len(X))

0.5
